# Demo: Interactive Widgets
A notebook to explore the ipywidgets interactive widgets with clojupyter kernel.

Requires clojupyter-0.3.3.-snapshot.

In [ ]:
(require '[clojupyter.widgets.alpha :as alpha])
(require '[clojupyter.widgets.ipywidgets :as ipy])
(require '[clojure.string :as s])

The original example, as posted on [github](https://github.com/clojupyter/clojupyter/issues/74).

In [ ]:
(def IA
    (let [myfun (fn [{:keys [a b]}]
                  (str a " + " b " = " (+ a b)))
          label (ipy/label)
          slider-1 (ipy/int-slider {:value (rand-int 100)})
          slider-2 (ipy/int-slider {:value (rand-int 100)})]
        (alpha/interactive label myfun {:a slider-1, :b slider-2})))
IA

## Simple Widgets
### Text Widgets
Let's build a text widget that returns the reverse upper case of the input string.
The function bellow achieves that:

In [ ]:
(def rev-upper (comp s/upper-case (partial reduce str) reverse))

In [ ]:
(rev-upper "My text")

Passing the fn directly to the widget does not work, because the fn passed to interactive needs to take a hash-map argument.

In [ ]:
(let [t (ipy/text {:value "Hello world"})
      label (ipy/label {:value "My text widget"})]
  (alpha/interactive label rev-upper {:value t}))

We can define a wrapper fn that has the required signature

In [ ]:
(defn mapper
  [f]
  (fn [{arg :arg}] (f arg)))

and pass it to interactive instead of the original fn.

In [ ]:
(let [t (ipy/text {:value "aroma"})
      label (ipy/label)]
  (alpha/interactive label (mapper rev-upper) {:arg t}))

Password widget is basically the same as text widget, except that the input is not echoed to the user.

In [ ]:
(let [t (ipy/password {:value "top secret pwd" :description "Password"})
      label (ipy/label)]
  (alpha/interactive label (mapper identity) {:arg t}))

In [ ]:
(let [t (ipy/textarea {:value "Some lengthy description" :description "Your Text Here:"})
      label (ipy/label)]
  (alpha/interactive label (mapper #(s/replace % " " "_")) {:arg t}))

Note: *The widget above doesn't allow to resize its width.
The same is true when using a python kernel, so it's not an issue with clojupyter.
The html element does, however, change its style attributes width & height when resizing it.
Tested in Firefox 74.0 (64-bit)*

In [ ]:
(let [t (ipy/combobox {:options ["blue" "black" "green" "yellow"]:description "Pick a color"})
      label (ipy/label)]
  (alpha/interactive label (mapper identity) {:arg t}))

In [ ]:
(ipy/html {:value "<p>Hello <b>World</b></p>" :placeholder "Some HTML" :description "Some HTML"})

Example taken from: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#HTML-Math

In [ ]:
(ipy/html-math {:value "Some math and <i>HTML</i>: \(x^2\) and $$\frac{x+1}{x-1}$$"})

Note: *Example above fails with error: __Unsupported escape character: \\(__*

____
### Boolean Widgets


In [ ]:
(let [w (ipy/checkbox {:description "Click Me" :value true})
      label (ipy/label)]
  (alpha/interactive label (mapper (partial str "My state: ")) {:arg w}))

In [ ]:
(let [w (ipy/toggle-button {:value false :description "Click to Activate" :icon "hand-pointer"})]
  (alpha/interactive (ipy/label) (mapper {true "Active" false "Inactive"}) {:arg w}))

In [ ]:
(ipy/valid {:value true :description "Valid!"})

___
### Numeric Widgets

In [ ]:
(let [w (ipy/int-slider {:description "Double this:" :value 20 :min -10 :max 20
                         :orientation :vertical}) ;; Slider accepts keys as options
      label (ipy/label)]
  (alpha/interactive label (mapper (partial * 2)) {:arg w}))

In [ ]:
(let [w (ipy/float-slider {:description "Square this:" :value (rand) :min 0 :max 1
                           :step 0.01 :readout_format ".2f"})
      label (ipy/label)]
  (alpha/interactive label (mapper (comp (partial format "%.3f") #(* % %))) {:arg w}))

In [ ]:
(let [w (ipy/float-log-slider {:value 2e5 :base 10 :min 4 :max 8 :step 0.2})]
  (alpha/interactive (ipy/label) (mapper (comp (partial format "%.2e") float (partial / 1))) {:arg w}))

Note: *The slider above kills the kernel for values > 1e7.*

In [ ]:
(let [w (ipy/int-range-slider {:value [200 1800] :max 3000 :step 5})]
  (alpha/interactive (ipy/label) (mapper #(/ (+ (first %) (last %)) 2)) {:arg w}))  

Note: *Passing the type fn to most (all?) widgets kills the kernel.*

In [ ]:
(ipy/float-range-slider)

In [ ]:
(ipy/int-progress {:min 10 :max 100 :value 95})

In [ ]:
(ipy/float-progress {:min 0 :max 10 :step 0.1 :value 4.8})

Note: *How does one animate the above progress bars?*

In [ ]:
(ipy/bounded-int-text {:min 0 :max 100 :step 5})

Note: *The above widget acts like a bounded-float-text if we pass a float step.*

In [ ]:
(ipy/bounded-float-text {:min 0 :max 10 :step 0.2})

In [ ]:
(ipy/int-text {:value 233})

In [ ]:
(ipy/float-text {:value 9.33 :step 0.02})

____
### Selection Widgets

In [ ]:
(ipy/dropdown {:options ["one" "two" "three"] :value "one"}) 

Note: *The above example fails. To fix it, one must pass the values to :_options_labels key.
The default value does not get passed to the widget.*

In [ ]:
(ipy/dropdown {:_options_labels ["one" "two" "three"] :value "one" :description "Pick a number"}) 

In [ ]:
(ipy/dropdown {:_options_labels ['("one" 1) '("two" 2) '("three" 3)] :value "one" :description "Pick a number"}) 

Note: *The above example does not work. Error: __Error displaying widget__*

In [ ]:
(ipy/radio-buttons {:options ["dark" "light"] :value "light"})

Note: *Just as dropdown, the widget fails when passing the values to :options and :values does not work as default values.*

In [ ]:
(ipy/radio-buttons {:_options_labels ["dark" "light"] :description "Theme"})

In [ ]:
(ipy/select {:options ["english" "french" "german"] :value "french"})

In [ ]:
(ipy/select {:_options_labels ["english" "french" "german"] :value "french" :description "Language"})

Note: *Just as dropdown, the widget fails when passing the values to :options and :values does not work as default values.*

In [ ]:
(ipy/selection-slider {:options ["slower" "slow" "normal" "fast" "fastest"] :value "french" :description "Speed"})

In [ ]:
(ipy/selection-slider {:_options_labels ["slower" "slow" "normal" "fast" "fastest"] :value "normal" :description "Speed"})

Note: *Just as dropdown, the widget fails when passing the values to :options and :values does not work as default values.*

In [ ]:
(ipy/selection-range-slider {:options (map list ["slower" "slow" "normal" "fast" "fastest"] (range)):value "french" :description "Speed"})

In [ ]:
(ipy/selection-range-slider {:_options_labels (map list ["slower" "slow" "normal" "fast" "fastest"] (range)):value "french" :description "Speed"})

Note: *Just as dropdown, the widget fails when passing the values to :options and :values does not work as default values.*

In [ ]:
(ipy/toggle-buttons {:options ["english" "french" "german"] :value "french" :description "Language"})

In [ ]:
(let [w (ipy/toggle-buttons {:_options_labels ["english" "french" "german"] :value "french" :description "Language"})]
  (alpha/interactive (ipy/label) (mapper identity) {:arg w}))  

Note: *Just as dropdown, the widget fails when passing the values to :options and :values does not work as default values.*
Note: *Clicking on any of the above buttons kills the kernel.*

In [ ]:
(ipy/select-multiple {:options ["Guitar" "Mandolin" "Violin" "Bass"] :value "Mandolin" :description "String Instruments"})

In [ ]:
(let [w (ipy/select-multiple {:_options_labels ["Guitar" "Mandolin" "Violin" "Bass"] :value "Mandolin" :description "String Instruments"})]
  (alpha/interactive (ipy/label) (mapper identity) {:arg w}))

Note: *Just as dropdown, the widget fails when passing the values to :options and :values does not work as default values.*
Note: *The above widget has has no output.*

____
### Container Widgets

In [ ]:
(let [items (map (comp ipy/label str) (range 4))]
  (ipy/box items))

Note: *The above example has no output.*

In [ ]:
(let [items (map (comp ipy/label str) (range 4))]
  (ipy/h-box items))

Note: *Same as box, the above example has no output.*

In [ ]:
(let [items (map (comp ipy/label str) (range 4))
      left-box (ipy/v-box [(first items) (second items)])
      right-box (ipy/v-box [(nth items 2) (last items)])]
  (ipy/h-box [left-box right-box]))

Note: *Same as box, the above example has no output.*

In [ ]:
#_(ipy/accordion {:children [ipy/int-slider ipy/text] :titles ["Slider" "Text"]})

Note: *The above example kills the kernel.*

In [ ]:
#_(ipy/tab {:children [ipy/int-slider ipy/text] :titles ["Slider" "Text"]})

Note: *Same as accordion, the above example kills the kernel.*

In [ ]:
#_(ipy/stacked [(ipy/button {:description "Click Me"}) (ipy/int-slider)])

Note: *Stacked is not implemented.*

____
### Other Widgets

In [ ]:
(require '[clojure.java.io :as io])
(defn slurp-bytes
  [^String filename & opt]
  (let [file (io/file filename)
        buf (byte-array (.length file))]
    (with-open [fis (io/make-input-stream file (when opt (apply hash-map opt)))]
      (.read fis buf))
    buf))

In [ ]:
(let [img (slurp-bytes "../resources/clojupyter/assets/logo-64x64.png")]
  (ipy/image {:value img :format :png :height 10 :width 10}))   

Note: *It looks like :width and :height don't change the output.*

In [ ]:
(let [w (ipy/button {:description "Click Here"})]
  (alpha/interactive (ipy/label) (mapper identity) {:arg w}))  

Note: *Clicking on the above button kills the kernel.*

In [ ]:
(let [min 0
      max 100
      value 30
      step 1
      w (ipy/play {:value value :min min :max max :step step :interval 500})
      prg (ipy/int-progress {:min min :max max :value value})]
  (alpha/interactive (ipy/h-box [w prg]) identity {:arg w}))    

Note: *As per [example](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html#Play-(Animation)-widget).
Currently, we are missing the jslink function to link two widgets.*

In [ ]:
(let [w (ipy/date-picker {:description "Pick a date" :disabled false})]
  (alpha/interactive (ipy/label) (mapper :year) {:arg w}))  

In [ ]:
(let [w (ipy/color-picker {:description "Pick a color" :value "blue" :concise true})]
  (alpha/interactive (ipy/label) (mapper identity) {:arg w}))

In [ ]:
(def fu (ipy/file-upload))
fu

Looking at the contents of fu atom we don't see any obvious way to access the contents of the uploaded file, only the metadata.

In [ ]:
@fu

In [ ]:
(ipy/controller)

Not tested: dom-widget, directional-link,  grid-box, layout, link, progress-style, slider-style, widget-display-data